# Understand and Reduce Bias in Machine Learning

Building accurate machine learning models is not always good enough, especially when predictions are used to make decisions that impact peoples lives. Models trained using supervised learning techniques attempt to find patterns or generalizations in a training dataset, then use those patterns to make predictions. These patterns may sometimess be undesirable or even illegal. For example, a loan risk model may determine that age plays a significant role in the prediction of a loan being deemed risky because the training dataset happened to have more risk for one age group than for another. 

This notebook gives practical examples to define and quantify the fairness of both data and models, exploring algorithms to detect bias and disparity in data, and mitigate bias in both data and models.

## 1.0 Bias Definitions

Although bias can have multiple definitions in the context of machine learning (i.e [**bias error where models miss relevant relations between features and targets**](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff), [**bias of an estimator**](https://en.wikipedia.org/wiki/Bias_of_an_estimator), etc). There are certain types of bias that can cause unfairness:

* **Selection bias** involves individuals being more likely to be selected for study than others, biasing the sample
* **Spectrum bias** arises from evaluating diagnostic tests on biased patient samples, leading to an overestimate of the sensitivity and specificity of the test
* The **bias of an estimator** is the difference between an estimator's expected value and the true value of the parameter being estimated
* **Omitted-variable bias** is the bias that appears in estimates of parameters in regression analysis when the assumed specification omits an independent variable that should be in the model
* **Detection bias** occurs when a phenomenon is more likely to be observed for a particular set of study subjects. 
* **Funding bias** may lead to the selection of outcomes, test samples, or test procedures that favor a study's financial sponsor.
* **Reporting bias** involves a skew in the availability of data, such that observations of a certain kind are more likely to be reported.
* **Analytical bias** arises due to the way that the results are evaluated.
* **Exclusion bias** arise due to the systematic exclusion of certain individuals from the study.
* **Attrition bias** arises due to a loss of participants e.g. loss to follow up during a study.
* **Recall bias** arises due to differences in the accuracy or completeness of participant recollections of past events. e.g. a patient cannot recall how many cigarettes they smoked last week exactly, leading to over-estimation or under-estimation.
* **Observer bias** arises when the researcher subconsciously influences the experiment due to cognitive bias where judgment may alter how an experiment is carried out / how results are recorded.

For fair models, metrics are needed that can help explore if any of the above biases is present in data and models. 

## 2.0 AI Fairness Toolkit

Throughout this notebook the [AI Fairness 360](https://github.com/Trusted-AI/AIF360) toolkit will be used. AI Fairness 360 (AIF360) is an extensible, open source toolkit for measuring, understanding, and removing AI bias. It contains the most widely used bias metrics, bias mitigation algorithms, and metric explainers from the top AI fairness researchers across industry & academia. The aif360 Python package includes a comprehensive set of metrics for datasets and models to test for biases, explanations for these metrics, and algorithms to mitigate bias in datasets and models. Find more resources and an interactive demo [here](http://aif360.mybluemix.net/).

### Import Packages

In [ ]:
!pip install --upgrade 'aif360[all]' --no-cache | tail -n 1
!pip install --upgrade cvxpy --no-cache | tail -n 1
!pip install --upgrade tqdm --no-cache | tail -n 1

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
sys.path.insert(1, "../")  

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

np.random.seed(0)

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_german
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions import get_distortion_german
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification

from IPython.display import Markdown, display
%matplotlib inline

## 3.0 Load and Explore Data

A [dataset](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29) that classifies people described by a set of attributes as good or bad credit risks.

This data is one of the example [datasets](https://aif360.readthedocs.io/en/latest/modules/datasets.html#module-aif360.datasets) used in aif360 and has it's own [class](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.GermanDataset.html#aif360.datasets.GermanDataset) that will be used. 

### Load data

It is assumed that the dataset can be found within a specific location. Let's create this folder and the download the data to this new folder. The advantage is that this works both locally and when running in Watson Studio.

In [ ]:
aif360_location = !python -c "from distutils.sysconfig import get_python_lib; print(get_python_lib())"
import os
install_loc = os.path.join(aif360_location[0], "aif360/data/raw/german/")
%cd $install_loc

In [ ]:
!wget ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.data
!wget ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.doc
%cd -

In [ ]:
dataset_german = GermanDataset()

### AIF 360 Data Format

All variables of this dataset are described in the [documentation](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.GermanDataset.html) with more details in the description of the [`StandardDataset`](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.StandardDataset.html). In short, the dataset class contains a numpy array or pandas DataFrame with several variables. 

In [ ]:
print(f'Features Type: {type(dataset_german.features)}')
print(f'Labels: {dataset_german.label_names}')
print(f'Protected Attributes: {dataset_german.protected_attribute_names}')
print(f'Number of Features: {len(dataset_german.feature_names)}')

In [ ]:
print(f'Feature Names: {dataset_german.feature_names}')

### Explore with pandas

Convert the data to a `features` DataFrame and `labels` Series.

Read more about the data and its attributes [here](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29).

In [ ]:
features = pd.DataFrame(dataset_german.features, columns=dataset_german.feature_names)
labels = pd.Series(dataset_german.labels.ravel(), name=dataset_german.label_names[0])

In [ ]:
features.describe().transpose().head(55)

### Feature Distribution

In [ ]:
plt.rcParams["figure.figsize"] = (18,18)

features.hist();
plt.tight_layout()

## 4.0 Exploring Bias with AIF360

When considering a model created in a supervised machine learning process:

<img src="https://nbviewer.jupyter.org/github/IBM/AIF360/blob/master/examples/images/Complex_NoProc_V3.jpg"   width="500" align="left">


Bias can enter the system in any of these three steps:
1. The process starts with a training dataset, which contains a sequence of instances, where each instance has two components: the features and the correct prediction for those features.
  * The training data set may be biased in that its outcomes may be biased towards particular kinds of instances
2. A machine learning algorithm is trained on this training dataset to produce a machine learning model. This generated model can be used to make a prediction when given a new instance.
  * The algorithm that creates the model may be biased in that it may generate models that are weighted towards particular features in the input
3. A second dataset with features and correct predictions, called a test dataset, is used to assess the accuracy of the model. Since this test dataset is the same format as the training dataset, a set of instances of features and prediction pairs, often these two datasets derive from the same initial dataset. A random partitioning algorithm is used to split the initial dataset into training and test datasets.
  * The test data set may be biased in that it has expectations on correct answers that may be biased

These three points in the machine learning process represent points for testing and mitigating bias. In AI Fairness 360 these are called:
* pre-processing 
* in-processing
* post-processing


### Bias Metrics and Definitions

To help understand bias, the AIF360 toolkit provides more than 70 metrics. The metrics you might choose to measure fairness will vary based on the use case or application being built, as well as where in the machine learning pipline you want to measure (i.e the training data or the model). [This demo](http://aif360.mybluemix.net/data) provides explanations and definitions of the metrics as well. Some intial guidance based on how you want to measure fairness:

- **Group Fairness**: Partitions a population into groups defined by protected attributes and seeks for some statistical measure to be equal across groups. For group fairness you might choose to use the metrics in the [DatasetMetric class](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.DatasetMetric.html). 
- **Individual Fairness**: Seeks for similar individuals to be treated similarly. For individual fairness you might choose to use the metrics in the [SampleDistortionMetric class](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.SampleDistortionMetric.html)
- **Both**: When considering both group and individual fairness, you might choose to explore the generalized entropy index and its specializations to Theil index and coefficient of variation in the [ClassificationMetric class](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.ClassificationMetric.html)


Before we calculate the metrics, some quick definitions:

* `mean_difference` (alias of `statistical_parity_difference`) - The difference of the rate of favorable outcomes received by the unprivileged group to the privileged group. A negative value indicates less favorable outcomes for the unprivileged groups. Although an ideal value of this metric is 0, fairness for the metric is between -0.1 and 0.1

* `disparate_impact` - The ratio of rate of favorable outcome for the unprivileged group to that of the privileged group
     
* `consistency` - The individual fairness metric that measures how similar the labels are for similar instances.

* `equal_opportunity_difference` (alias of `true_positive_rate_difference`) 
    * true_positive_rate: ratio of true positives to positive examples in the dataset

* `base_rate` - The number of positives divided by number of positives plus negatives


### 4.1 Bias in Credit Dataset (Training Data)

In the dataset we are exploring, bias could occur based on age or sex. To explore the age bias:

* set the protected attribute to be `age`, where `age >=25` is considered privileged
* the protected attribute for `sex` is not consider in this evaluation
* split the original dataset into training and testing datasets
* set two variables for the privileged (1) and unprivileged (0) values for the age attribute. These are key inputs for detecting and mitigating bias



The protected attribute will be "Age", with "1" (older than or equal to 25) and "0" (younger than 25) being the values for the privileged and unprivileged groups, respectively.

#### Metrics based on a single `BinaryLabelDataset`

Since the dataset we are exploring has two values of the label, we can look for [binary dataset label metrics](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.BinaryLabelDatasetMetric.html)

In [ ]:
dataset_german = GermanDataset(protected_attribute_names=['age'],
                    privileged_classes=[lambda x: x >= 25],      
                    features_to_drop=['personal_status', 'sex']) 

# Split into train, validation, and test
dataset_german_train, dataset_german_test = dataset_german.split([0.7], shuffle=True)

# Age: 1 are those older than or equal to 25. 0 is younger than 25
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

In [ ]:
metric_german_train = BinaryLabelDatasetMetric(dataset_german_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

metric_german_test = BinaryLabelDatasetMetric(dataset_german_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

#help(metric_german_train)

In [ ]:
print("Metrics for Original Training Dataset:")
print("\tDifference in mean outcomes between unprivileged and privileged groups = %f" % metric_german_train.mean_difference())
print("\tdisparate_impact = %f" % metric_german_train.disparate_impact())
print("\tconsistency = %f" % metric_german_train.consistency())
print("\tbase_rate = %f" % metric_german_train.base_rate())
print("\tnum_negatives = %f" % metric_german_train.num_negatives())
print("\tnum_positives = %f" % metric_german_train.num_positives())

#print("Metrics for Original Test dataset:")
#print("\tmean_difference = %f" % metric_german_test.mean_difference())
#print("\tdisparate_impact = %f" % metric_german_test.disparate_impact())


### 4.2 Bias in Credit Model (ML Algorithm)

Before we can explore the bias in the machine learning algorithm. We need to build the model. When we explored the dataset we have loaded, you may have noticed:
- It is fairly clean and tidy (no missing values)
- It is already one-hot encoded for multiple classes

The only thing we really need to do is scale and normalize the features. For this we can use [StandardScaler](https://scikit-learn.org/stable/modules/preprocessing.html) - 
*Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn; they might behave badly if the individual features do not more or less look like standard normally distributed data: Gaussian with zero mean and unit variance. `StandardScaler` implements the Transformer API to compute the mean and standard deviation on a training set so as to be able to later reapply the same transformation on the testing set.*

aif360 format can be used with scikitlearn!

In [ ]:
scale_german = StandardScaler().fit(dataset_german_train.features)

X_train = scale_german.transform(dataset_german_train.features)
y_train = dataset_german_train.labels.ravel()
w_train = dataset_german_train.instance_weights.ravel()

X_test = scale_german.transform(dataset_german_test.features)
y_test = dataset_german_test.labels.ravel()
w_test = dataset_german_test.instance_weights.ravel()

# plt.rcParams["figure.figsize"] = (18,18)
# scaled_features = pd.DataFrame(X_train, columns=dataset_german.feature_names)
# scaled_features.hist();
# plt.tight_layout()

#### Build Model Using Original Training Data

There are a variety of algorithms we could use for this binary classification scenario. For this exercise, we will simply use Logistic Regression and then use the model to make predicitons on the test dataset.

In [ ]:
# Logistic regression classifier and predictions

# create an instance of the model
lmod = LogisticRegression()

# train the model
lmod.fit(X_train, y_train, 
         sample_weight=dataset_german_train.instance_weights)

# calculate predicted labels
y_train_pred = lmod.predict(X_train)

# assign positive class index
pos_ind = np.where(lmod.classes_ == dataset_german_train.favorable_label)[0][0]

# add predicted labels to predictions dataset
dataset_german_train_pred = dataset_german_train.copy()
dataset_german_train_pred.labels = y_train_pred

dataset_german_test_pred = dataset_german_test.copy(deepcopy=True)
X_test = scale_german.transform(dataset_german_test_pred.features)
y_test = dataset_german_test_pred.labels
dataset_german_test_pred.scores = lmod.predict_proba(X_test)[:,pos_ind].reshape(-1,1)


#### Model Performance


In [ ]:
score = lmod.score(X_test, y_test)
print("Logistic Regression model accuracy = %f" % score)

# confusion matrix
cm = metrics.confusion_matrix(y_test, lmod.predict(X_test))

plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');

In [ ]:
df_test = pd.DataFrame(dataset_german_test.features, columns=dataset_german_test.feature_names)

[fig, ax] = plt.subplots(1,2, figsize=(15, 5));
plot_confusion_matrix(lmod, X_test[df_test['age']==0], y_test[df_test['age']==0],
                      cmap=plt.cm.Blues, 
                      display_labels=['good credit','bad credit'],
                      ax=ax[0]);
ax[0].set_title('Age < 25')

plot_confusion_matrix(lmod, X_test[df_test['age']==1], y_test[df_test['age']==1],
                      cmap=plt.cm.Blues, 
                      display_labels=['good credit','bad credit'],
                      ax=ax[1]);
ax[1].set_title('Age > 25');

#### Metrics based on the `ClassificationMetric class`

Since we have built a classification model, we can explore the [`ClassificationMetric class metrics`](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.ClassificationMetric.html) for bias in predictions


In [ ]:
# Calculate the metrics on both the training and test data
metric_german_train_pred = ClassificationMetric(dataset_german_train, dataset_german_train_pred,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

metric_german_pred = ClassificationMetric(dataset_german_test, dataset_german_test_pred,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

In [ ]:
print("Predicted labels (training dataset):")
print("\tmean_difference = %f" % metric_german_train_pred.mean_difference())
print("\tdisparate_impact = %f" % metric_german_train_pred.disparate_impact())

print("Predicted labels (test dataset):")
print("\tmean_difference = %f" % metric_german_pred.mean_difference())
print("\tdisparate_impact = %f" % metric_german_pred.disparate_impact())

print("Quality metrics  (training dataset):")
print("\taccuracy = %f" % metric_german_train_pred.accuracy())
print("\tcoefficient_of_variation = %f" % metric_german_train_pred.coefficient_of_variation())
print("\terror_rate = %f" % metric_german_train_pred.error_rate())

print("Quality metrics  (test dataset):")
print("\taccuracy = %f" % metric_german_pred.accuracy())
print("\tcoefficient_of_variation = %f" % metric_german_pred.coefficient_of_variation())
print("\terror_rate = %f" % metric_german_pred.error_rate())


## 5.0 AI Fairness Algorithms

There are three points in the machine learning process where testing and mitigating bias can be done. In AI Fairness 360 these are called:
- pre-processing (i.e mitigation happens before the creation of the model).
- in-processing
- post-processing

### 5.1 Pre-processing algorithms

The previous sections showed that the privileged group was getting 17% more positive outcomes in the training dataset. Since this is not desirable, we are going to try to mitigate this bias in the training dataset. AI Fairness 360 implements several pre-processing mitigation algorithms. 

We will choose the Reweighing algorithm [1], which is implemented in the Reweighing class in the aif360.algorithms.preprocessing package. This algorithm will transform the dataset to have more equity in positive outcomes on the protected attribute for the privileged and unprivileged groups. We then call the fit and transform methods to perform the transformation, producing a newly transformed training dataset (dataset_transf_train).

[1] F. Kamiran and T. Calders, "Data Preprocessing Techniques for Classification without Discrimination," Knowledge and Information Systems, 2012.

#### Remove Bias by Reweighing Data

**Reweighing** is a preprocessing technique that weights the examples in each (group, label) combination differently to ensure fairness before classification. Read the [documentation](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.preprocessing.Reweighing.html) for a full overview.

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)

# compute the weights for reweighing the dataset
RW.fit(dataset_german_train)

# transform the dataset to a new dataset based on the estimated transformation
dataset_rw_train = RW.transform(dataset_german_train)
dataset_rw_test = RW.transform(dataset_german_test)

In [ ]:
print("Metrics for Original Training Dataset:")
print("\tDifference in mean outcomes between unprivileged and privileged groups = %f" % metric_german_train.mean_difference())
print("\tdisparate_impact = %f" % metric_german_train.disparate_impact())
#print("\tconsistency = %f" % metric_german_train.consistency())
#print("\tbase_rate = %f" % metric_german_train.base_rate())
#print("\tnum_negatives = %f" % metric_german_train.num_negatives())
#print("\tnum_positives = %f" % metric_german_train.num_positives())
#print("\tsmoothed_empirical_differential_fairness = %f" % metric_german_train.smoothed_empirical_differential_fairness())

metric_rw_train = BinaryLabelDatasetMetric(dataset_rw_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)

print("Metrics for Reweighted Training Dataset:")
print("\tDifference in mean outcomes between unprivileged and privileged groups = %f" % metric_rw_train.mean_difference())
print("\tdisparate_impact = %f" % metric_rw_train.disparate_impact())
#print("\tconsistency = %f" % metric_rw_train.consistency())
#print("\tbase_rate = %f" % metric_rw_train.base_rate())
#print("\tnum_negatives = %f" % metric_rw_train.num_negatives())
#print("\tnum_positives = %f" % metric_rw_train.num_positives())
#print("\tsmoothed_empirical_differential_fairness = %f" % metric_rw_train.smoothed_empirical_differential_fairness())

#### Build Model Using Reweighted Training Data

We can now train a new machine learning model using the reweighted training data. For this exercise, we will simply use Logistic Regression and then use the model to make predicitons on the test dataset.

In [ ]:
# scale data
scale_rw = StandardScaler().fit(dataset_rw_train.features)

X_train_rw = scale_rw.transform(dataset_rw_train.features)
y_train_rw = dataset_rw_train.labels.ravel()
w_train_rw = dataset_rw_train.instance_weights.ravel()

X_test_rw = scale_rw.transform(dataset_rw_test.features)
y_test_rw = dataset_rw_test.labels.ravel()
w_test_rw = dataset_rw_test.instance_weights.ravel()

#dataset_rw_train.instance_weights

In [ ]:
# create a new instance of the model
lmod_rw = LogisticRegression()

# train the model
lmod_rw.fit(X_train_rw, y_train_rw, 
         sample_weight=dataset_rw_train.instance_weights)

# calculate predicted labels
y_train_pred_rw = lmod_rw.predict(X_train_rw)

# assign positive class index
pos_ind_rw = np.where(lmod_rw.classes_ == dataset_rw_train.favorable_label)[0][0]

# add predicted labels to predictions dataset
dataset_rw_train_pred = dataset_rw_train.copy()
dataset_rw_train_pred.labels = y_train_pred_rw

dataset_rw_test_pred = dataset_rw_test.copy()
X_test = scale_rw.transform(dataset_rw_test_pred.features)
y_test_pred = lmod.predict(X_test)
pos_ind_test = np.where(lmod_rw.classes_ == dataset_rw_test.favorable_label)[0][0]
dataset_rw_test_pred.labels = lmod_rw.predict(X_test)

In [ ]:
# model accuracy
print("Original Logistic Regression model accuracy = %f" % score)
score_rw = lmod_rw.score(X_test_rw, y_test_rw)
print("Reweighted Logistic Regression model accuracy = %f" % score_rw)

#### Metrics based on the `ClassificationMetric class`

Since we have built a classification model, we can explore the [`ClassificationMetric class metrics`](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.ClassificationMetric.html) for bias in predictions

In [ ]:
metric_german_train_pred_rw = ClassificationMetric(dataset_rw_train, dataset_rw_train_pred,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

metric_german_pred_rw = ClassificationMetric(dataset_rw_test, dataset_rw_test_pred,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)


In [ ]:
print("Original test dataset:")
print("\tmean_difference = %f" % metric_german_test.mean_difference())
print("\tdisparate_impact = %f" % metric_german_test.disparate_impact())

print("Predicted labels (test dataset):")
print("\tmean_difference = %f" % metric_german_pred.mean_difference())
print("\tdisparate_impact = %f" % metric_german_pred.disparate_impact())

print("Predicted labels (reweighted test dataset):")
print("\tmean_difference = %f" % metric_german_pred_rw.mean_difference())
print("\tdisparate_impact = %f" % metric_german_pred_rw.disparate_impact())

print("Quality metrics  (test dataset):")
print("\taccuracy = %f" % metric_german_pred.accuracy())
print("\tcoefficient_of_variation = %f" % metric_german_pred.coefficient_of_variation())
print("\terror_rate = %f" % metric_german_pred.error_rate())

print("Quality metrics  (reweighted test dataset):")
print("\taccuracy = %f" % metric_german_pred_rw.accuracy())
print("\tcoefficient_of_variation = %f" % metric_german_pred_rw.coefficient_of_variation())
print("\terror_rate = %f" % metric_german_pred_rw.error_rate())



In [ ]:
[fig, ax] = plt.subplots(2,2, figsize=(15, 12));
plot_confusion_matrix(lmod, X_test[df_test['age']==0], y_test[df_test['age']==0],
                      cmap=plt.cm.Blues, 
                      display_labels=['good credit','bad credit'],
                      ax=ax[0,0]);
ax[0,0].set_title('Age < 25')

plot_confusion_matrix(lmod, X_test[df_test['age']==1], y_test[df_test['age']==1],
                      cmap=plt.cm.Blues, 
                      display_labels=['good credit','bad credit'],
                      ax=ax[0,1]);
ax[0,1].set_title('Age > 25');

plot_confusion_matrix(lmod_rw, X_test_rw[df_test['age']==0], y_test_rw[df_test['age']==0],
                      cmap=plt.cm.Blues, 
                      display_labels=['good credit','bad credit'],
                      ax=ax[1,0]);
ax[1,0].set_title('Age < 25 (Reweighted)')

plot_confusion_matrix(lmod_rw, X_test_rw[df_test['age']==1], y_test_rw[df_test['age']==1],
                      cmap=plt.cm.Blues, 
                      display_labels=['good credit','bad credit'],
                      ax=ax[1,1]);
ax[1,1].set_title('Age > 25 (Reweighted)');

### Other debiasing Algorithms

There are too many algorithms and points in the process we can look to mitigate bias to explore in this lab. Some other debiasing algorithsm include:

- Another example of pre-processing debiasing is the optimized data pre-processing algorithm. The debiasing function used is implemented in the [OptimPreproc](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.preprocessing.OptimPreproc.html?highlight=get%20distortion) class. It modifies training data features & labels. See the example notebook here: https://github.com/Trusted-AI/AIF360/blob/master/examples/demo_optim_data_preproc.ipynb

- There are several in-processing algorithms. Including:
  - [Adversarial Debiasing](https://github.com/Trusted-AI/AIF360/blob/master/examples/demo_adversarial_debiasing.ipynb) - Uses adversarial techniques to maximize accuracy & reduce evidence of protected attributes in predictions
  - [Reject Option Classification](https://github.com/Trusted-AI/AIF360/blob/master/examples/demo_reject_option_classification.ipynb) - Adds a discrimination-aware regularization term to the learning objective
  
- There are several post-processing algorithms. Including:
  - [Reject option classification (ROC)](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.postprocessing.RejectOptionClassification.html?highlight=reject) is a postprocessing technique that gives favorable outcomes to unpriviliged groups and unfavorable outcomes to priviliged groups in a confidence band around the decision boundary with the highest uncertainty.

Another full example: https://nbviewer.jupyter.org/github/IBM/AIF360/blob/master/examples/tutorial_medical_expenditure.ipynb

## Acknowledgements and Credits

This notebook is adapted from several examples & tutorials:

- https://github.com/IBMDeveloperUK/Trusted-AI-Workshops
- https://github.com/Trusted-AI/AIF360/tree/master/examples
- https://nbviewer.jupyter.org/github/IBM/AIF360/blob/master/examples/tutorial_credit_scoring.ipynb
    
Copyright © 2021 IBM. This notebook and its source code are released under the terms of the MIT License.